In [2]:
import sys
import pathlib
sys.path.append("..")

from parse_concept_net import ParseConceptNet

In [24]:
pcn = ParseConceptNet()

with open('../Samples/mary.txt', 'r') as file:
    for line in file:
        print(line)
#     story = file.read()
    
# file.close()
# print(story)